In [3]:
from azureml.core import Workspace

ws = Workspace.from_config()
print("✅ Azure ML Workspace Loaded:", ws.name)


✅ Azure ML Workspace Loaded: mlops-simple-flow


In [4]:
from azureml.core.model import Model

model = Model.register(
    workspace=ws,
    model_name="catboost-thrombosis-predictor",
    model_path="catboost_model.cbm",  # Path to your model file
    description="CatBoost model for predicting thrombotic events",
)

print("✅ Model registered successfully!")


Registering model catboost-thrombosis-predictor
✅ Model registered successfully!


In [3]:
env_yaml = """name: catboost-env
dependencies:
  - python=3.8
  - pip:
    - azureml-defaults
    - catboost
    - joblib
    - numpy
    - pandas
"""
with open("environment.yml", "w") as f:
    f.write(env_yaml)

print("✅ environment.yml created successfully!")


✅ environment.yml created successfully!


In [5]:
from azureml.core import Environment

env = Environment.from_conda_specification(name="catboost-env", file_path="environment.yml")
env.register(workspace=ws)

print("✅ Environment registered successfully!")


✅ Environment registered successfully!


In [5]:
score_script = """import json
import numpy as np
import pandas as pd
import joblib
from azureml.core.model import Model
from catboost import CatBoostClassifier

def init():
    global model
    model_path = Model.get_model_path("catboost-thrombosis-predictor")
    model = CatBoostClassifier()
    model.load_model(model_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)["data"]
        df = pd.DataFrame(data)
        
        # Convert categorical features to int if necessary
        categorical_features = ["gender", "work_type", "Residence_type", "smoking_status"]
        for col in categorical_features:
            if col in df.columns:
                df[col] = df[col].astype(str).astype("category")

        predictions = model.predict(df)
        return {"predictions": predictions.tolist()}
    except Exception as e:
        return {"error": str(e)}
"""

with open("score.py", "w") as f:
    f.write(score_script)

print("✅ score.py created successfully!")


✅ score.py created successfully!


In [7]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

# Define inference configuration
inference_config = InferenceConfig(
    entry_script="score.py",  # The script handling inference requests
    environment=env
)

# Define ACI deployment configuration
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy the model
service_name = "catboost-endpoint"
service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config
)

service.wait_for_deployment(show_output=True)
print(f"✅ Model deployed successfully! Endpoint URL: {service.scoring_uri}")


/tmp/ipykernel_5655/1950278768.py:15: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-02-01 15:24:01+00:00 Creating Container Registry if not exists.
2025-02-01 15:24:02+00:00 Building image..
2025-02-01 15:33:01+00:00 Generating deployment configuration.
2025-02-01 15:33:02+00:00 Submitting deployment to compute..
2025-02-01 15:33:09+00:00 Checking the status of deployment catboost-endpoint..
2025-02-01 15:34:45+00:00 Checking the status of inference endpoint catboost-endpoint.
Succeeded
ACI service creation operation finished, operation "Succeeded"
✅ Model deployed successfully! Endpoint URL: http://57c1d30e-f22e-414a-811a-0c6f2e757de0.westeurope.azurecontainer.io/score


In [14]:
from azureml.core import Workspace, Model, Environment
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# ✅ Load Azure ML workspace
ws = Workspace.from_config()

# ✅ Load the correct environment (Fixes the issue)
env = Environment.get(workspace=ws, name="catboost-env")

# ✅ Define inference configuration (Using the correct environment object)
inference_config = InferenceConfig(
    entry_script="score.py",
    environment=env  # ✅ Corrected
)

# ✅ Define deployment configuration
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# ✅ Deploy the model
service = Model.deploy(
    workspace=ws,
    name="catboost-endpoint",
    models=[Model(ws, "catboost-thrombosis-predictor")],
    inference_config=inference_config,
    deployment_config=aci_config
)

# ✅ Wait for deployment to complete
service.wait_for_deployment(show_output=True)

# ✅ Print the endpoint URL
print(f"✅ Model deployed successfully! Endpoint URL: {service.scoring_uri}")


/tmp/ipykernel_5655/1865427786.py:21: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-02-01 16:19:13+00:00 Creating Container Registry if not exists.
2025-02-01 16:19:14+00:00 Use the existing image.
2025-02-01 16:19:14+00:00 Generating deployment configuration.
2025-02-01 16:19:16+00:00 Submitting deployment to compute.
2025-02-01 16:19:22+00:00 Checking the status of deployment catboost-endpoint..
2025-02-01 16:20:48+00:00 Checking the status of inference endpoint catboost-endpoint.
Succeeded
ACI service creation operation finished, operation "Succeeded"
✅ Model deployed successfully! Endpoint URL: http://a4258b93-e2e4-429e-b794-1da10226599d.westeurope.azurecontainer.io/score


In [15]:
# Replace with actual endpoint
endpoint = service.scoring_uri
print(endpoint)

http://a4258b93-e2e4-429e-b794-1da10226599d.westeurope.azurecontainer.io/score


In [19]:
import requests
import json

# ✅ Replace with your actual endpoint
endpoint = service.scoring_uri  

# ✅ Ensure categorical features are **strings**
test_data = {
    "data": [
        {
            "gender": "1",  # ✅ Convert to string
            "age": 60,  # ✅ Keep numeric features as float
            "hypertension": "1",
            "heart_disease": "0",
            "ever_married": "1",
            "work_type": "2",
            "Residence_type": "1",
            "avg_glucose_level": 85.6,
            "bmi": 28.9,
            "smoking_status": "0"
        }
    ]
}

headers = {"Content-Type": "application/json"}

# ✅ Send request & get response
response = requests.post(endpoint, data=json.dumps(test_data), headers=headers)
print("🚀 Response from Model:", response.json())


🚀 Response from Model: {'error': 'Invalid type for cat_feature[non-default value idx=0,feature_idx=0]=1.0 : cat_features must be integer or string, real number values and NaN values should be converted to string.'}


In [17]:
# import requests
# import json

# endpoint = service.scoring_uri  

# # Ensure the test data follows the model’s expected feature names & types
# test_data = {
#     "data": [
#         {
#             "gender": 1,
#             "age": 60,
#             "hypertension": 1,
#             "heart_disease": 0,
#             "ever_married": 1,
#             "work_type": 2,
#             "Residence_type": 1,
#             "avg_glucose_level": 85.6,
#             "bmi": 28.9,
#             "smoking_status": 0
#         }
#     ]
# }

# headers = {"Content-Type": "application/json"}

# # Send request & get response
# response = requests.post(endpoint, data=json.dumps(test_data), headers=headers)
# print("🚀 Response from Model:", response.json())


🚀 Response from Model: {'error': 'Invalid type for cat_feature[non-default value idx=0,feature_idx=0]=1.0 : cat_features must be integer or string, real number values and NaN values should be converted to string.'}
